In [2]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [3]:
def adjust_mean(timeseries):
    n=len(timeseries)
    decay=0.99
    weights=np.ones(n)/float(n)
    line=np.array([1.0001**(-decay*(n-t)) for t in np.linspace(1,n,num=n)])
    adjust_weight=line*weights
    return np.sum(adjust_weight*timeseries)

In [4]:
def read_values(data_list):
    allFiles = glob.glob( "../values/*.csv")
    for name in allFiles:
        data_list[name[10:-4]]=pd.read_csv(name)

In [5]:
def sharp_ratio(df,adjust=False):
    vals=df['net_worth'].values
    hourly_return=np.log(vals[1:]/vals[0:-1])
    if adjust==False:
        volatility=np.std(hourly_return)
        mean=np.mean(hourly_return)
    else:
        mean=adjust_mean(hourly_return)
        volatility=np.power(adjust_mean(np.square(np.array(hourly_return-mean))),0.5)
    return mean/volatility*8760/np.sqrt(8760.0)
        
    

In [6]:
def max_drawdown(df,adjust=False):
    vals=df['net_worth'].values
    n=len(vals)
    i = np.argmax(np.maximum.accumulate(vals) - vals)
    j = np.argmax(vals[:i])
    decay=0.99
    date_end=df.iloc[i]['date']
    date_start=df.iloc[j]['date']
    if adjust==False:
        return ((float(vals[i]) / vals[j]) - 1), date_start, date_end
    else:
        return ((float(vals[i]) / vals[j]) - 1)*1.0001**(-decay*(n-(i+j)/2)), date_start, date_end

In [7]:
def days_trading(df):
    duration=df['date'].values
    first_day=datetime.strptime(duration[0], "%Y-%m-%d %H:%M:%S")
    last_day=datetime.strptime(duration[-1], "%Y-%m-%d %H:%M:%S")
    return (last_day-first_day).days*24.0+(last_day-first_day).seconds/3600.0
        

In [8]:
def prob_up(df):
    vals=df['net_worth'].values
    hourly_return=np.log(vals[1:]/vals[0:-1])
    temp1=[x>0 for x in hourly_return]
    temp2=[x<0 for x in hourly_return]
    return np.sum(temp1)/float(np.sum(temp1)+np.sum(temp2))

In [9]:
def cumulate_return(df):
    vals=df['net_worth'].values
    return (vals[-1]/vals[0]-1)

In [10]:
def ret_drawdown(df):
    vals=df['net_worth'].values
    mean_ret=(vals[-1]/vals[0]-1)/len(vals)
    return -mean_ret/max_drawdown(df)[0]

In [11]:
def basic_analysis(data_list):
    value_return={}
    for name,data in data_list.iteritems():
        value_return[name]=(sharp_ratio(data),sharp_ratio(data,True),-max_drawdown(data)[0],-max_drawdown(data,True)[0],
                            prob_up(data),cumulate_return(data),ret_drawdown(data),days_trading(data))
    return value_return

In [12]:
if __name__ == '__main__':
    data_list={} #all dataframes
    read_values(data_list)
    performance=basic_analysis(data_list)
    sort_sharp=sorted(performance.items(),key=lambda x: x[1][0] ,reverse=True)
    sort_adjust_sharp=sorted(performance.items(),key=lambda x: x[1][1] ,reverse=True)
    sort_drawdown=sorted(performance.items(),key=lambda x: x[1][2])
    sort_adjust_drawdown=sorted(performance.items(),key=lambda x: x[1][3])
    sort_prob_up=sorted(performance.items(),key=lambda x: x[1][4],reverse=True)
    sort_return=sorted(performance.items(),key=lambda x: x[1][5],reverse=True)
    sort_ret_drawdown=sorted(performance.items(),key=lambda x: x[1][6],reverse=True)
    for item in sort_drawdown:
        print item[0], item[1][2]
    print '\n'
    for item in sort_adjust_drawdown:
        print item[0], item[1][3]
    analysis_df=pd.DataFrame.from_dict(performance,orient='index')
    analysis_df.columns=['sharp','adjusted_sharp','max_drawdown','adjust_drawdown',
                         'prob_up','cumulate_return','ret_drawdown','hours_trading']
    analysis_df.to_csv('./analysis.csv')

OI 0.010819182603
pp 0.0115733245913
p 0.0143180146209
v 0.014804957169
c 0.0171919770801
ZC 0.0210756633608
TA 0.0211973321165
l 0.0226680834274
MA 0.0230468995405
jm 0.0233914093562
cs 0.0261346689551
y 0.0304042199116
j 0.0307676246498
jd 0.0370049505018


v 0.00450300331919
c 0.00576231601843
pp 0.00608079852417
OI 0.00686501668243
cs 0.00885734311287
MA 0.00907562135262
l 0.00920098920261
y 0.0094194528299
jm 0.00993792884462
TA 0.0105587001186
p 0.0112021954158
jd 0.0126273672105
j 0.0134444634715
ZC 0.0167392706944
